# checking the order that it reads in the h5 files!

In [2]:
import h5py
import glob
import numpy as np
from astropy.table import Table
from Payne.fitting.fitutils import airtovacuum
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation
import astropy.units as u
from scipy import constants
speedoflight = constants.c / 1000.0

In [4]:
def getdata(catfile=None,gaiaid=None,starind=None):
    th5 = h5py.File(catfile,'r')
    cat_i = Table(th5['catalog'][()])

    if gaiaid != None:
        t = th5[f'{gaiaid}']
        cat_ii = cat_i[cat_i['GAIAEDR3_ID'] == gaiaid]
        cat = {x:cat_ii[x][0] for x in cat_ii.keys()}

    elif starind != None:
        cat_ii = cat_i[starind]
        cat = {x:cat_ii[x] for x in cat_ii.keys()}
        
        gaiaid = cat['GAIAEDR3_ID']
        t = th5[f'{gaiaid}']
                
    header = {ii:th5['header'][ii][()] for ii in th5['header'].keys()}
    hdr_date = header['DATE-OBS']
    location = EarthLocation.of_site('MMT')

    sc = SkyCoord(ra=cat['GAIAEDR3_RA']*u.deg,dec=cat['GAIAEDR3_DEC']*u.deg)
    barycorr = sc.radial_velocity_correction(obstime=Time(hdr_date), location=location)
    HC = float(barycorr.to(u.km/u.s).value)

    out = {}
    out['spec'] = {}
    out['phot'] = {}
    
    # spec
    out['spec']['obs_wave']  = (1.0 - (HC/speedoflight)) * airtovacuum(t['wave'][()])
    out['spec']['obs_flux']  = t['flux'][()]
    out['spec']['obs_eflux'] = t['eflux'][()]

    # medflux = np.nanmedian(out['spec']['obs_flux'])
    # out['spec']['obs_flux'] = out['spec']['obs_flux'] / medflux
    # out['spec']['obs_eflux'] = out['spec']['obs_eflux'] / medflux

    SNR = np.nanmedian(out['spec']['obs_flux']/out['spec']['obs_eflux'])
    out['spec']['SNR'] = SNR
    out['spec']['date'] = hdr_date
    out['spec']['HC'] = HC

    # phot
    phot = t['phot']
    filterarr = list(phot.keys())
    usedfilters = []
    
    for ff in filterarr:            
        phot_i = phot[ff][()]
        if np.isfinite(phot_i[0]) & (phot_i[1] > 0.0):
            if 'Gaia' in ff:
                ff = ff.replace('EDR3', 'DR3')
            if 'PS_y' in ff:
                print(f"Skipping filter {ff}")
                continue
            out['phot'][ff] = [phot_i[0],phot_i[1]]
            usedfilters.append(ff)
    out['phot_filtarr'] = usedfilters
    
    # parallax
    out['parallax'] = [cat['GAIAEDR3_PARALLAX'],cat['GAIAEDR3_PARALLAX_ERROR']]
    
    # define some cluster guesses
    if 'ic348' in catfile:
        Avest = 1.91 # Cantat-Gaudin+ 2020
        RVest = 15.44 # Tarricq+ 2021
    elif 'm37' in catfile:
        Avest = 0.75
        RVest = 8.81
    elif 'm3' in catfile:
        Avest = 0.03 # Harris 1996
        RVest = -147.2 # BAUMGARDT H. and HILKER M 2018
    elif 'm67' in catfile:
        Avest = 0.07
        RVest = 34.18
    elif 'ngc6791' in catfile:
        Avest = 0.70
        RVest = -47.75
    elif 'ngc6811' in catfile:
        Avest = 0.09
        RVest = 7.17
    elif 'ngc6819' in catfile:
        Avest = 0.40
        RVest = 2.80
    elif 'ngc6866' in catfile:
        Avest = 0.48
        RVest = 12.44
    else:
        print('Could not understand cluster name in filename, setting Avest = 0.1 and RVest = 0.0')
        Avest = 0.1
        RVest = 0.0
    
    out['Avest'] = Avest
    out['RVest'] = RVest

    out['Teff']   = 5770.0
    out['[Fe/H]'] = 0.0
    out['[a/Fe]'] = 0.0
    out['log(g)'] = 4.5
    out['log(R)'] = 0.0

    return out

In [9]:
def getall(gaiaid=None,date=None,cluster=None):
    
    datasrcstr ='/data/labs/douglaslab/sofairj/data/hectochelle_rereduction/'
    
    filesrstr = '*'
    # list of all possible HDF5 files
    if cluster != None:
        filesrstr += f'{cluster}*'
    if date != None:
        filesrstr += f'{date}*'

    filesrstr += '.h5'
    flist = glob.glob(datasrcstr+filesrstr)

    # get the HDF5 files with this star
    workingfiles = []
    for ff in flist:
        with h5py.File(ff,'r') as th5:
            if f'{gaiaid}' in list(th5.keys()):
                workingfiles.append(ff)
    
    if len(workingfiles) == 0:
        print(f'Could Not Find Any Spectra for GaiaID = {gaiaid}')
        return None
    print(f'here are the working files: {workingfiles}')

    out = {}
    out['spec'] = []
    out['specname'] = []
    for ii,ww in enumerate(workingfiles):
        data = getdata(catfile=ww,gaiaid=gaiaid)
        if ii == 0:
            out['phot']     = data['phot']
            out['phot_filtarr'] = data['phot_filtarr']
            out['parallax'] = data['parallax']
            out['Avest']    = data['Avest']
            out['RVest']    = data['RVest']

            out['Teff']   = data['Teff']
            out['[Fe/H]'] = data['[Fe/H]']
            out['[a/Fe]'] = data['[a/Fe]']
            out['log(g)'] = data['log(g)']
            out['log(R)'] = data['log(R)']

        if data['spec']['SNR'] >= 3.0:
            out['spec'].append(data['spec'])
            out['specname'].append(ww.split('/')[-1])

    print(f'before if len out spec >2, here is out:\n{out}')
    
    if len(out['spec']) > 2:
        # sort spectra by date
        sortind = np.argsort(out['specname'])
        out['spec'] = list(np.array(out['spec'])[sortind])
        out['specname'] = list(np.array(out['specname'])[sortind])

    return out

In [11]:
getall_result = getall(gaiaid=2076392838230907392,cluster='ngc6819')

here are the working files: ['/data/labs/douglaslab/sofairj/data/hectochelle_rereduction/data_ngc6819_2010.0921_ngc6819_sep2010_1.7137.h5', '/data/labs/douglaslab/sofairj/data/hectochelle_rereduction/data_ngc6819_2011.0714_ngc6819_sep2010_2.1909.h5', '/data/labs/douglaslab/sofairj/data/hectochelle_rereduction/data_ngc6819_2011.0713_ngc6819_sep2010_1.1860.h5']
Skipping filter PS_y
Skipping filter PS_y
Skipping filter PS_y
before if len out spec >2, here is out:
{'spec': [{'obs_wave': array([5161.66345823, 5161.69958131, 5161.73570434, ..., 5291.59916915,
       5291.6314766 , 5291.66378218]), 'obs_flux': array([0.96012807, 0.92655271, 0.93196105, ..., 1.07549007, 1.08380928,
       1.01990492]), 'obs_eflux': array([0.06620144, 0.05900009, 0.05919646, ..., 0.06290141, 0.06317589,
       0.05997551]), 'SNR': 17.157727487873576, 'date': b'2010-09-21T05:50:36', 'HC': -11.176998605987093}, {'obs_wave': array([5161.35556738, 5161.39135747, 5161.42714751, ..., 5291.31511517,
       5291.346850

In [12]:
getall_result['specname']

['data_ngc6819_2010.0921_ngc6819_sep2010_1.7137.h5',
 'data_ngc6819_2011.0713_ngc6819_sep2010_1.1860.h5',
 'data_ngc6819_2011.0714_ngc6819_sep2010_2.1909.h5']